In [14]:
#pip install pydicom
#pip uninstall -y enum34
#pip install labelme==3.16.7
import cv2
import json
import numpy as np
import os
import torch
from Image_Segmentation.network import U_Net, AttU_Net
from PIL import Image
from pydicom import dcmread
from torchvision import transforms as T

In [6]:
value_to_label_name = {}
with open("classes.txt") as f:
    for index, line in enumerate(f):
        value_to_label_name[index] = line[:-1]
print(value_to_label_name)

n_classes = len(value_to_label_name)
print("loading segmentation model")
net = AttU_Net(img_ch=3, output_ch=n_classes).to('cuda')
net.load_state_dict(torch.load('Image_Segmentation/models/AttU_Net-f4-490-0.923.pkl'))
net.train(False)
print("done")

empty_response = {  "version": "4.5.6",
                    "flags": {},
                    "shapes": [],
                    "imagePath": "",
                    "imageData": None,
                    "imageHeight": 0,
                    "imageWidth": 0,
                    "shapes": [],
                 }

{0: '0. background', 1: '1. Rt psoas major mucle', 2: '2. Lt psoas major mucle', 3: '3. Rt quardratus lumborum muscle', 4: '4. Lt quardratus lumborum muscle', 5: '5. Rt eretor spinae muscle', 6: '6. Lt eretor spinae muscle', 7: '7. Rt trans abd,int and ext obl', 8: '8. Lt trans abd,int and ext obl', 9: '9. Rt rectus abdominus muscle', 10: '10. Lt rectus abdominus muscle', 11: '11.L3 Vertebral body'}
loading segmentation model
done


In [17]:
dcm_file_name = 'sarcopenia_data/SARCOPANIA0010.dcm'
file_name = dcm_file_name.split('.')[0].split('/')[-1]

ds = dcmread(dcm_file_name)
image = ds.pixel_array
image_h, image_w = image.shape
image = image.astype('float64')
intercept = ds.RescaleIntercept
wc = 50
ww = 250
UL = wc + ww/2
LL = wc - ww/2
slope = ds.RescaleSlope
image -= (-intercept+LL)
image[image<0] = 0
image[image>(UL-LL)] = UL-LL
image *= 255.0/image.max()
image = image.astype('uint8')
image = Image.fromarray(image)
image_src = image.convert('RGB')
image_src.save(file_name+'.png')
print(f"source image {file_name+'.png'} saved")

Transform = []
Transform.append(T.ToTensor())
Transform.append(T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)))
Transform = T.Compose(Transform)
image = Transform(image_src)
image = image.unsqueeze(0).to('cuda')

pred = torch.sigmoid(net(image))
pred = pred.squeeze(0).detach().cpu()
pred = np.argmax(pred, axis=0)

shapes = []
for i in range(1, n_classes):
    mask = pred[pred==i]
    layer = np.array(pred==i, np.uint8)*255 
    contours, _ = cv2.findContours(layer, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        contour = cv2.approxPolyDP(contour, 3, True).reshape(-1, 2).tolist()
        if len(contour) < 3: continue
        shape_data = {"label": value_to_label_name[i],
                      "points": contour,
                      "group_id": None,
                      "shape_type": "polygon",
                      "flags": {},
                     }
        shapes.append(shape_data)
    #plt.show()

empty_response["version"] = "4.5.6"
empty_response["imagePath"] = file_name+".png"
empty_response["imageHeight"] = image_h
empty_response["imageWidth"] = image_w
empty_response["shapes"] = shapes

with open(file_name+'.json', 'w') as outfile:
    json.dump(empty_response, outfile)
print(f"predict json {file_name+'.json'} saved")

source image SARCOPANIA0010.png saved
predict json SARCOPANIA0010.json saved
